# Flow and transport problem - SPE10

In this tutorial we investigate the transport problem where the advective field is computed with a Darcy model.

Before creating the grid we import NumPy, the SciPy sparse library and PorePy.

In [1]:
import numpy as np
import porepy as pp
import scipy.sparse as sps

import os

cwd_folder = os.getcwd()
spe10_folder = cwd_folder + "/spe10/"

import sys

sys.path.insert(1, spe10_folder)

from spe10 import Spe10

We specify number of cells in each dimension and the physical size of the domain. Then we create a Cartesian grid and compute geometric properties such as face centers, cell volumes etc.

In [2]:
selected_layers = 35  # 3, 35

# Define the class with the corresponding layer(s)
spe10 = Spe10(selected_layers)
# For simplicity we extract the grid form the class spe10
sd = spe10.sd

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
perm_dict = spe10.perm_as_dict()

We declare the data for the Darcy problem.

In [4]:
# Permeability
perm = pp.SecondOrderTensor(
    kxx=perm_dict["kxx"], kyy=perm_dict["kyy"], kzz=perm_dict["kzz"]
)

# Boundary conditions
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]

# define outflow and inflow type boundary conditions, left and right boundary
outflow = np.isclose(b_face_centers[1, :], spe10.full_physdims[1])
inflow = np.isclose(b_face_centers[1, :], 0)

# define the labels and values for the boundary faces
labels = np.array(["neu"] * b_faces.size)
labels[np.logical_or(inflow, outflow)] = "dir"

bc_val = np.zeros(sd.num_faces)
bc_val[b_faces[inflow]] = 1e8

bc = pp.BoundaryCondition(sd, b_faces, labels)

# Collect all parameters in a dictionary
parameters = {"second_order_tensor": perm, "bc": bc, "bc_values": bc_val}

We now set the data for the Darcy problem

In [5]:
flow_key = "flow"
flow_data = pp.initialize_default_data(sd, {}, flow_key, parameters)

We now solve the Darcy problem by using the MPFA scheme.

In [6]:
# construct the lhr and rhs from the discretization of the diffusion operator
mpfa = pp.Mpfa(flow_key)
mpfa.discretize(sd, flow_data)
A, b = mpfa.assemble_matrix_rhs(sd, flow_data)

# solve the problem
cell_p = sps.linalg.spsolve(A, b)

# now data contains the discretization matrices build from MPFA
mat_discr = flow_data[pp.DISCRETIZATION_MATRICES][flow_key]

q = mat_discr["flux"] @ cell_p + mat_discr["bound_flux"] @ bc_val

And we export the corresponding solutions.

In [7]:
# to export the flux
mvem = pp.MVEM(flow_key)
mvem.discretize(sd, flow_data)

# construct the P0 flux reconstruction
cell_q = mvem.project_flux(sd, q, flow_data)

save = pp.Exporter(sd, "sol_p", folder_name="ex5")

data_to_export = [
    ("kxx", np.log10(perm_dict["kxx"])),
    ("kyy", np.log10(perm_dict["kyy"])),
    ("kzz", np.log10(perm_dict["kzz"])),
    ("cell_p", cell_p),
    ("cell_q", cell_q),
]
save.write_vtu(data_to_export)

We now consider the transport problem where now the advective field is the one computed from the Darcy problem. First we set the data.

In [8]:
# Transport problem
transport_key = "transport"
delta_t = 0.001
num_steps = 40

# Set in the data file the flux
bc_val = np.zeros(sd.num_faces)
bc_val[b_faces[inflow]] = 1

parameters = {"darcy_flux": q, "bc": bc, "bc_values": bc_val}
transport_data = pp.initialize_default_data(sd, {}, transport_key, parameters)

As done in the other cases, we now construct the upwind matrix and the mass matrix.

In [9]:
# create the upwind and mass matrices
upwind = pp.Upwind(transport_key)

# discretize and get the matrices
upwind.discretize(sd, transport_data)

U, b_upwind = upwind.assemble_matrix_rhs(sd, transport_data)
M = sps.diags(sd.cell_volumes)

Finally, by using the implicit Euler we compute the concentration that is transported in the porous medium

In [10]:
# Initial condition and exporter
c = np.zeros(sd.num_cells)
save = pp.Exporter(sd, "sol_c", folder_name="ex5")
save.write_vtu([("conc", c)], time_step=0)

# IE with LU factorization
S = M + delta_t * U
lu = sps.linalg.splu(S.tocsc())
for i in np.arange(num_steps):
    c = lu.solve(M @ c - delta_t * b_upwind)
    print(np.amin(c), np.amax(c))
    save.write_vtu([("conc", c)], time_step=(i + 1) * delta_t)

# export the main pvd file
time = np.arange((num_steps + 1)) * delta_t
save.write_pvd(time)

5.402641987338601e-25 0.9822760258465867
8.351687998804009e-24 0.9996858607401692
6.78703607781227e-23 0.9999944322038373
3.8605403162813333e-22 0.999999901316484
1.7266814814145773e-21 0.9999999982508951
6.468244747487023e-21 0.9999999999689582
2.1110273600735774e-20 0.9999999999994089
6.165620870455716e-20 1.0000000000000695
1.6429132628598028e-19 1.0000000000001117
4.0521914622306286e-19 1.000000000000124
9.355372396448986e-19 1.000000000000434
2.0397495418814356e-18 1.000000000000519
4.230059003711767e-18 1.0000000000006897
8.393096803360259e-18 1.0000000000007068
1.6011358542914927e-17 1.0000000000007079
2.948846892046292e-17 1.000000000000708
5.261561631274973e-17 1.000000000000708
9.122621471633541e-17 1.000000000000708
1.540971833725105e-16 1.000000000000708
2.5416829714512713e-16 1.000000000000708
4.101659573409352e-16 1.000000000000708
6.487345571668601e-16 1.000000000001669
1.0071974515918731e-15 1.0000000000019065
1.537090459743027e-15 1.0000000004181966
2.3086338889076086e